In [1]:
import os
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

from get_density import get_density
from get_spins import get_spins
from downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds

In [2]:
box_N = 128
box_L = 100.0
def verify_box_spec(sim_path):
	config_path = os.path.join(sim_path, "simConfig.mat")
	simConfig = spio.loadmat(config_path)["simConfig"]
	if box_L != float(simConfig["Lbox"][0, 0][0, 0]):
		raise Exception(ValueError())
	if box_N != int(simConfig["N"][0, 0][0, 0]):
		raise Exception(ValueError())


VOL_SCALE_FACTOR = 8
vol_grid_axis = np.linspace(-box_L / 2, box_L / 2, num=box_N//VOL_SCALE_FACTOR)
(GX, GY, GZ) = [gi.flatten() for gi in np.meshgrid(vol_grid_axis, vol_grid_axis, vol_grid_axis)]
	

In [3]:
def load_snaps_at(sim_path, progress_indices):
	snap_paths = get_snap_paths(sim_path)
	outs = []
	for pr in progress_indices:
		ind = int(pr * (len(snap_paths) - 1))
		snap_path = snap_paths[ind]
		outs.append(load_Psi(snap_path))
	return outs


In [4]:
def load_Psi(snap_path):
	print(f"loading {snap_path}")
	snap = spio.loadmat(snap_path)
	Psi = np.stack([snap["Psi"][0, i] for i in range(3)], axis=0)
	print(f"Psi loaded")
	return Psi

def get_snap_paths(sim_path):
	verify_box_spec(sim_path)
	sim_files = os.listdir(sim_path)
	searcher = r"snap-Psi-(\d+)-(\d+\.\d*)\.mat"
	snap_files = [re.search(searcher, fname) for fname in sim_files]
	snap_files = [(int(f.groups()[0]), float(f.groups()[1])) for f in snap_files if not(f is None)]
	snap_files = sorted(snap_files)
	snap_files = [f"snap-Psi-{iter}-{time}.mat" for (iter, time) in snap_files]
	snap_paths = [os.path.join(sim_path, f) for f in snap_files]
	return snap_paths


In [5]:
def plot_snap_density(Psi):
	Rho = get_density(Psi)
    
	Val = downscale_3d(Rho, VOL_SCALE_FACTOR)
	plot = pgo.Volume(
		x = GX,
		y = GY,
		z = GZ,
		value = np.log1p(Val).flatten() / 3,
		opacity = 0.02,
		surface_count = 128,
		coloraxis = "coloraxis"
	)
	return plot

In [6]:
import math

klin = np.linspace(-box_N/2, box_N/2 - 1, box_N) * (2 * math.pi / box_L)
k_sq_nz = np.fft.fftshift(sum([np.square(k) for k in np.meshgrid(klin, klin, klin)]))
k_sq_nz[k_sq_nz == 0] = 1

def get_normalized_grav_potential(Rho):
	V_grav = np.fft.fftn(-Rho)
	V_grav = V_grav / k_sq_nz
	V_grav = np.real(np.fft.ifftn(V_grav))
	min_V = np.min(V_grav)
	max_V = np.max(V_grav)
	return (V_grav - min_V) / (max_V - min_V)

def plot_snap_V_grav(Psi):
	Rho = get_density(Psi)
	V_grav = get_normalized_grav_potential(Rho)
	return pgo.Volume(
		x = GX,
		y = GY,
		z = GZ,
		value = 1 - downscale_3d(V_grav, VOL_SCALE_FACTOR).flatten(),
		opacity = 0.02,
		surface_count = 128,
		coloraxis = "coloraxis"
	)

In [7]:
def plot_snap_spins(Psi):
	Rho = get_density(Psi)
	Spins = get_spins(Psi)
	rt3 = 3**0.5
	Spp = [downscale_3d(Spin / Rho, VOL_SCALE_FACTOR).flatten() / rt3 for Spin in Spins]
	return pgo.Cone(
		x=GX, y=GY, z=GZ,
		u=Spp[0], v=Spp[1], w=Spp[2],
		coloraxis = "coloraxis"
	)

In [ ]:
Psis = load_snaps_at(f"../outputs/2022-07-20/8-solitons-random-128-attractive-run-1/", [0.0, 0.5, 1.0])
plots = [
	[plot_snap_density(Psi) for Psi in Psis],
	[plot_snap_V_grav(Psi) for Psi in Psis],
	[plot_snap_spins(Psi) for Psi in Psis]
]
fig, js = plot_multiple_3Ds(plots)
fig.update_layout(
	coloraxis = {
		'colorscale': 'jet',
		'cmin': 0.0,
		'cmax': 1.0,
	},
)
fig.write_html("plot_Rho_Spin.html", post_script=js)

In [ ]:
# counts, bin_edges = np.histogram(V_grav.flatten(), bins = 256, weights = Rho.flatten())
# fig = pgo.Figure(data=[
# 	pgo.Bar(x=bin_edges, y=counts)
# ])
# fig.write_html("plot_of_V_grav.html", include_plotlyjs='cdn')

In [8]:
def get_spin_norm_dist(Psi):
	Rho = get_density(Psi)
	V_grav = get_normalized_grav_potential(Rho)
	Spins = get_spins(Psi)
	Spin_norm = np.sqrt(np.sum(np.square(Spins), axis=0))
	V_grav_f = V_grav.flatten()
	sorter = np.argsort(V_grav_f)
	V_grav_s = V_grav_f[sorter]
	# Rho_f = Rho.flatten()
	# sorter = np.argsort(Rho)
	# Rho_s = Rho_f[sorter]
	Rho_s = Rho.flatten()[sorter]
	Spin_norm_s = Spin_norm.flatten()[sorter]
	NUM_BINS = 192
	spin_hist, spin_edges = np.histogram(V_grav_s, bins=NUM_BINS, weights=Spin_norm_s)
	mass_hist, mass_edges = np.histogram(V_grav_s, bins=NUM_BINS, weights=Rho_s)
	assert(np.all(spin_edges == mass_edges))
	spin_cumu = np.cumsum(spin_hist)
	mass_cumu = np.cumsum(mass_hist)
	avrg_cumu = spin_cumu / mass_cumu
	return pgo.Scatter(
		x=spin_edges, y=avrg_cumu,
		mode = "lines+markers",
	)

	# return average_spin_norm_in(Psi, True), average_spin_norm_in(Psi, V_grav <= 1/3)
	# print(average_spin_in(Psi, True, 0))
	# print(average_spin_in(Psi, V_grav <= 0.3, 0))
	# for j in range(3):
	# 	print(f"axis: {j}		entire box: {average_spin_in(Psi, True, j):.4f}		dense area: {average_spin_in(Psi, V_grav <= 1/3, j):.4f}")
	
	# sp = get_spins(Psi) / Rho
	# sps = np.sum(np.square(sp), axis=0)
	# counts, edges = np.histogram(sps.flatten(), bins=256)
	# pgo.Figure(data=[
	# 	pgo.Bar(x=edges, y=counts)
	# ]).show()

In [10]:
from plotly.subplots import make_subplots
NUM_SIMULATIONS = 1
fig = make_subplots(NUM_SIMULATIONS, 1, shared_xaxes=True)

DISPLAY_TYPES = [
	# filename,		 label,				 color
	("nosi",		"No SI",			"#1abc9c"),
	("attractive",	"Attractive SI",	"#3498db"),
	("repulsive",	"Repulsive SI",		"#f39c12"),
]

for i in range(1, 1+NUM_SIMULATIONS):
	for (filename, label, color) in DISPLAY_TYPES:
		Psis = load_snaps_at(f"../outputs/2022-07-25/8-solitons-random-144-{filename}-run-{i}/", [1.0])
		trace = get_spin_norm_dist(Psis[-1])
		trace.name = label
		trace.line.color = color
		fig.add_trace(trace, i, 1)

fig = pgo.Figure(fig)
fig.write_html("plot_Spin_hist_new.html")

loading ../outputs/2022-07-25/8-solitons-random-144-nosi-run-1/snap-Psi-12000-6367.27.mat
Psi loaded
loading ../outputs/2022-07-25/8-solitons-random-144-attractive-run-1/snap-Psi-12000-6367.27.mat
Psi loaded
loading ../outputs/2022-07-25/8-solitons-random-144-repulsive-run-1/snap-Psi-12000-6367.27.mat
Psi loaded
